# 01. Setup y Carga de Datos M5

---

**Fase 1** - Setup y Exploración de Datos

**Objetivo:** Cargar y explorar los datos M5 de Kaggle, comprender su estructura y preparar el dataset base agregado **por producto** para análisis de forecasting de urgencias.

**Enfoque de manufactura:** Agregamos por producto (sumando todas las tiendas) porque desde perspectiva de fabricación lo que importa es **cuánto producir de cada producto en total**, independientemente de a qué tienda se envíe.

**Input:** 
- `data/raw/sales_train_evaluation.csv` (ventas diarias por producto y tienda)
- `data/raw/calendar.csv` (mapeo de días a fechas)
- `data/raw/sell_prices.csv` (precios por tienda y fecha)

**Output:**
- `data/processed/sales_daily_by_product.csv` (ventas diarias agregadas por producto)
- `data/processed/sales_weekly_by_product.csv` (ventas semanales por producto)

---

## 1. Configuración Inicial

In [ ]:
# Imports estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
np.random.seed(42)

# Estilo de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')

print("✓ Librerías cargadas correctamente")
print(f"  - Pandas version: {pd.__version__}")
print(f"  - NumPy version: {np.__version__}")

In [ ]:
# Configuración de paths
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
FIGURES = PROJECT_ROOT / 'results' / 'figures'

# Crear carpetas si no existen
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
FIGURES.mkdir(parents=True, exist_ok=True)

# Constantes del proyecto
RANDOM_SEED = 42
FIGSIZE_STANDARD = (12, 6)
FIGSIZE_WIDE = (15, 5)

print(f"✓ Paths configurados:")
print(f"  - DATA_RAW: {DATA_RAW}")
print(f"  - DATA_PROCESSED: {DATA_PROCESSED}")
print(f"  - FIGURES: {FIGURES}")

## 2. Carga de Datos M5

### Contexto del Dataset M5

El dataset M5 (Makridakis-5) de Kaggle contiene datos de ventas de Walmart con:
- **3,049 productos únicos** (item_id)
- **10 tiendas** en 3 estados (CA, TX, WI)
- **30,490 series** (productos × tiendas)
- **1,941 días** de ventas históricas

**Estrategia de agregación:**
Agregaremos por `item_id` (producto) sumando todas las tiendas, porque desde perspectiva de manufactura:
- Lo que importa es la **demanda total de cada producto**
- La distribución a tiendas es logística posterior
- Resultado: 3,049 series temporales (una por producto)

In [ ]:
# Cargar calendar
print("Cargando calendar.csv...")
calendar = pd.read_csv(DATA_RAW / 'calendar.csv')
calendar['date'] = pd.to_datetime(calendar['date'])

print(f"✓ Calendar cargado: {calendar.shape}")
print(f"  - Rango de fechas: {calendar['date'].min()} a {calendar['date'].max()}")
print(f"  - Total días: {len(calendar)}")
print()

calendar.head()

In [ ]:
# Cargar sales
print("Cargando sales_train_evaluation.csv...")
print("(Puede tardar 10-20 segundos)")

sales = pd.read_csv(DATA_RAW / 'sales_train_evaluation.csv')

print(f"✓ Sales cargado: {sales.shape}")
print(f"  - Series (producto × tienda): {sales.shape[0]:,}")
print(f"  - Días de ventas: {sales.shape[1] - 6}")
print(f"  - Productos únicos: {sales['item_id'].nunique()}")
print(f"  - Tiendas: {sales['store_id'].nunique()}")
print()

sales.head()

## 3. Agregación por Producto

### 3.1 Sumar Ventas de Todas las Tiendas por Producto

Vamos a crear dataset con ventas agregadas por `item_id`, sumando todas las tiendas.

In [ ]:
print("Agregando ventas por producto (sumando todas las tiendas)...")
print("(Esta operación puede tardar 20-30 segundos)")
print()

# Extraer columnas de días
day_cols = [col for col in sales.columns if col.startswith('d_')]

# Agrupar por item_id y sumar ventas de todas las tiendas
sales_by_product = sales.groupby('item_id')[day_cols].sum().reset_index()

print(f"✓ Agregación completada")
print(f"  - Productos: {len(sales_by_product)}")
print(f"  - Días: {len(day_cols)}")
print(f"  - Shape: {sales_by_product.shape}")
print()

print("Primeros productos:")
sales_by_product.head()

In [ ]:
# Estadísticas de ventas por producto
product_stats = pd.DataFrame({
    'item_id': sales_by_product['item_id'],
    'total_sales': sales_by_product[day_cols].sum(axis=1),
    'mean_daily': sales_by_product[day_cols].mean(axis=1),
    'std_daily': sales_by_product[day_cols].std(axis=1),
    'cv': sales_by_product[day_cols].std(axis=1) / sales_by_product[day_cols].mean(axis=1)
})

print("Estadísticas por producto:")
print("="*60)
print(product_stats.describe())
print()

# Top 10 productos más vendidos
print("Top 10 productos más vendidos:")
print(product_stats.nlargest(10, 'total_sales')[['item_id', 'total_sales', 'mean_daily', 'cv']])

### 3.2 Transformar a Formato Largo

In [ ]:
print("Transformando a formato largo...")
print("(Esta operación puede tardar 30-60 segundos)")
print()

# Melt: convertir columnas d_1, d_2, ... a filas
sales_long = sales_by_product.melt(
    id_vars=['item_id'],
    var_name='d',
    value_name='sales'
)

print(f"✓ Transformación completada: {sales_long.shape}")
print(f"  - Registros: {len(sales_long):,}")
print()

sales_long.head(10)

In [ ]:
# Merge con calendar para obtener fechas e información temporal
print("Agregando información temporal desde calendar...")

sales_long = sales_long.merge(
    calendar[['d', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year']],
    on='d',
    how='left'
)

print(f"✓ Merge completado: {sales_long.shape}")
print()

sales_long.head(10)

### 3.3 Agregación Semanal por Producto

Agregamos a nivel semanal para reducir ruido y facilitar forecasting.

In [ ]:
print("Agregando a nivel semanal por producto...")
print("(Esta operación puede tardar 20-40 segundos)")
print()

# Agrupar por producto y semana
sales_weekly = sales_long.groupby(['item_id', 'wm_yr_wk']).agg({
    'date': 'min',  # Primer día de la semana
    'sales': 'sum'   # Total de ventas semanales
}).reset_index()

# Renombrar columnas
sales_weekly.columns = ['item_id', 'week_id', 'week_start', 'total_sales']

# Ordenar por producto y fecha
sales_weekly = sales_weekly.sort_values(['item_id', 'week_start']).reset_index(drop=True)

print(f"✓ Agregación semanal completada")
print(f"  - Total registros: {len(sales_weekly):,}")
print(f"  - Productos: {sales_weekly['item_id'].nunique()}")
print(f"  - Semanas por producto: ~{len(sales_weekly) / sales_weekly['item_id'].nunique():.0f}")
print()

sales_weekly.head(15)

In [ ]:
# Añadir week_num como índice secuencial por producto
sales_weekly['week_num'] = sales_weekly.groupby('item_id').cumcount()

# Estadísticas semanales por producto
print("Estadísticas de ventas semanales:")
print("="*60)
print(sales_weekly['total_sales'].describe())
print()

# Productos con más ventas semanales promedio
top_products_weekly = sales_weekly.groupby('item_id')['total_sales'].mean().nlargest(10)
print("Top 10 productos por ventas semanales promedio:")
print(top_products_weekly)

## 4. Visualización de Series Temporales

### 4.1 Visualizar Top Productos

In [ ]:
# Seleccionar top 5 productos por volumen total
top5_products = (sales_weekly.groupby('item_id')['total_sales']
                  .sum()
                  .nlargest(5)
                  .index.tolist())

print(f"Top 5 productos por volumen total:")
for i, prod in enumerate(top5_products, 1):
    total = sales_weekly[sales_weekly['item_id'] == prod]['total_sales'].sum()
    print(f"  {i}. {prod}: {total:,.0f} unidades")

In [ ]:
# Visualizar series temporales de top 5 productos
fig, axes = plt.subplots(5, 1, figsize=(15, 15))

colors = ['#2E86AB', '#A23B72', '#06A77D', '#F18F01', '#C73E1D']

for i, (prod, color) in enumerate(zip(top5_products, colors)):
    data = sales_weekly[sales_weekly['item_id'] == prod]
    
    axes[i].plot(data['week_start'], data['total_sales'], 
                linewidth=1.5, color=color, marker='o', markersize=2)
    axes[i].axhline(data['total_sales'].mean(), color='red', 
                   linestyle='--', linewidth=1, alpha=0.6,
                   label=f"Media: {data['total_sales'].mean():.0f}")
    
    axes[i].set_title(f'Producto: {prod}', fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Unidades')
    axes[i].legend(loc='upper right')
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)

axes[-1].set_xlabel('Fecha')

plt.tight_layout()
plt.savefig(FIGURES / '01_series_temporales_top_productos.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_series_temporales_top_productos.png")

### 4.2 Distribución de Ventas por Producto

In [ ]:
# Distribución de ventas semanales agregadas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de ventas semanales
axes[0].hist(sales_weekly['total_sales'], bins=100, color='#2E86AB', alpha=0.7, edgecolor='black')
axes[0].axvline(sales_weekly['total_sales'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f"Media: {sales_weekly['total_sales'].mean():.0f}")
axes[0].set_title('Distribución de Ventas Semanales por Producto', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Unidades Vendidas')
axes[0].set_ylabel('Frecuencia')
axes[0].set_yscale('log')  # Escala log para ver mejor
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(sales_weekly['total_sales'], vert=True)
axes[1].set_title('Boxplot de Ventas Semanales', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Unidades Vendidas')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES / '01_distribucion_ventas_productos.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_distribucion_ventas_productos.png")

## 5. Validación de Datos

In [ ]:
print("Ejecutando validaciones de calidad de datos...")
print("="*60)
print()

# 1. Missing values
print("1. MISSING VALUES:")
missing = sales_weekly.isnull().sum()
if missing.sum() == 0:
    print("   ✓ No hay valores nulos")
else:
    print("   ⚠ Valores nulos encontrados:")
    print(missing[missing > 0])
print()

# 2. Valores negativos
print("2. VALORES NEGATIVOS:")
negative_sales = (sales_weekly['total_sales'] < 0).sum()
if negative_sales == 0:
    print("   ✓ No hay ventas negativas")
else:
    print(f"   ⚠ {negative_sales} registros con ventas negativas")
print()

# 3. Productos con pocas ventas
print("3. PRODUCTOS DE BAJO VOLUMEN:")
sales_by_prod = sales_weekly.groupby('item_id')['total_sales'].sum()
low_volume = (sales_by_prod < 100).sum()  # Menos de 100 unidades en todo el período
print(f"   - Productos con <100 ventas totales: {low_volume}")
print(f"   - Productos con >0 ventas: {(sales_by_prod > 0).sum()}")
print()

# 4. Completitud temporal
print("4. COMPLETITUD TEMPORAL:")
weeks_per_product = sales_weekly.groupby('item_id').size()
print(f"   - Semanas mínimas por producto: {weeks_per_product.min()}")
print(f"   - Semanas máximas por producto: {weeks_per_product.max()}")
print(f"   - Todos los productos tienen mismas semanas: {weeks_per_product.nunique() == 1}")
print()

print("="*60)
print("✓ Validación completada")

## 6. Guardar Datos Procesados

In [ ]:
# Optimizar tipos de datos
print("Optimizando tipos de datos...")

sales_weekly['week_num'] = sales_weekly['week_num'].astype('int16')
sales_weekly['total_sales'] = sales_weekly['total_sales'].astype('int32')

print("✓ Tipos optimizados")

In [ ]:
# Guardar datasets procesados
print("Guardando datos procesados...")
print()

# 1. Sales semanales por producto (principal para forecasting)
output_file = DATA_PROCESSED / 'sales_weekly_by_product.csv'
sales_weekly.to_csv(output_file, index=False)

print(f"✓ sales_weekly_by_product.csv guardado")
print(f"  - Registros: {len(sales_weekly):,}")
print(f"  - Productos: {sales_weekly['item_id'].nunique()}")
print(f"  - Tamaño: {output_file.stat().st_size / 1024:.2f} KB")
print()

print("="*60)
print("✓ Datos procesados guardados en: data/processed/")

## 7. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo
n_products = sales_weekly['item_id'].nunique()
n_weeks = sales_weekly['week_num'].max() + 1
total_records = len(sales_weekly)

print(f"""
╔═══════════════════════════════════════════════════════════════════╗
║                    RESUMEN EJECUTIVO - FASE 1                     ║
╚═══════════════════════════════════════════════════════════════════╝

📊 DATOS CARGADOS:
  • Dataset M5 (Walmart Sales) de Kaggle
  • Agregación por PRODUCTO (sumando todas las tiendas)
  • Enfoque de manufactura: demanda total por producto

📈 AGREGACIÓN FINAL:
  • {n_products} productos únicos
  • {n_weeks} semanas por producto
  • {total_records:,} registros totales (producto × semana)
  • Período: {sales_weekly['week_start'].min()} a {sales_weekly['week_start'].max()}

📊 ESTADÍSTICAS DE VENTAS SEMANALES:
  • Promedio por producto-semana: {sales_weekly['total_sales'].mean():.1f} unidades
  • Mediana: {sales_weekly['total_sales'].median():.1f} unidades
  • Desviación estándar: {sales_weekly['total_sales'].std():.1f} unidades
  • Coef. variación: {sales_weekly['total_sales'].std() / sales_weekly['total_sales'].mean():.2f}

✅ CALIDAD DE DATOS:
  • ✓ Sin valores nulos
  • ✓ Sin ventas negativas
  • ✓ Series temporales completas por producto
  • ✓ {n_products} series listas para análisis de urgencias

📁 OUTPUTS GENERADOS:
  • data/processed/sales_weekly_by_product.csv
  • 2 visualizaciones en results/figures/

🎯 PRÓXIMOS PASOS (Fase 2):
  1. Detección de urgencias por producto (threshold: desviaciones estándar)
  2. Análisis de variabilidad por producto
  3. Simulación de urgencias sintéticas
  4. Análisis de múltiples series temporales

═══════════════════════════════════════════════════════════════════
✓ Fase 1 completada exitosamente
═══════════════════════════════════════════════════════════════════
""")

---

## Notas Técnicas

### Decisiones Tomadas:

1. **Agregación por producto (item_id):** Se suma ventas de todas las tiendas porque desde perspectiva de manufactura lo relevante es la demanda total a producir, no la distribución geográfica.

2. **Nivel semanal:** Reduce ruido diario y proporciona horizonte de planificación relevante para manufactura.

3. **3,049 series temporales:** Cada producto es una serie independiente. Esto permite:
   - Detectar urgencias específicas por producto
   - Modelar patrones diferentes entre productos
   - Planificación granular de producción

4. **Sin filtrado de productos:** Se mantienen todos los productos, incluyendo bajo volumen, para capturar urgencias en productos de baja rotación (pueden ser más críticas).

### Próxima Fase:

En Fase 2 usaremos **threshold basado en desviaciones estándar** (~1.4-1.5 σ) para detectar urgencias:
- Urgencia = ventas > (μ + k×σ) donde k ≈ 1.4-1.5
- Estadísticamente más robusto que threshold sobre media móvil
- Adaptable a la variabilidad específica de cada producto

---

**Siguiente notebook:** `02_simulacion_urgencias_eda.ipynb`